In [40]:
import pandas as pd
import glob
import json
import os
import ast
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from keras.layers import Dense,Conv1D,Flatten,MaxPool1D
from keras.models import Sequential
import matplotlib.pyplot as plt

In [41]:
path = '/content/drive/MyDrive/Dataset/angle/'
dir_list = os.listdir(path)

print(dir_list)

['sit', 'walk', 'jump', 'clap', 'wave', 'stand', 'run']


In [42]:
def extract_angles(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    angle_list = []
    for frame in data["data"]:
        for angle in frame["angle"]["data"]:
            angle_list.append(float(angle["angle"]))
    return angle_list

In [43]:
all_rows = []
for action in dir_list:
    rows = []
    path = '/content/drive/MyDrive/Dataset/angle/'+action
    file = glob.glob(path+'/*.json')
    for filename in file:
        angle = extract_angles(filename)
        row = {'angles': angle, 'action': action}
        rows.append(row)
    rows = np.array(rows)
    all_rows.extend(rows)
data = pd.DataFrame(all_rows)    
data

,angles,action
0,"[88.56790381583536, 117.5528115767178, 90.0, 1...",sit
1,"[40.10090754621224, 46.16913932790743, 88.5311...",sit
2,"[81.02737338510362, 351.0273733851036, 68.1985...",sit
3,"[92.04540848888723, 85.23635830927383, 117.407...",sit
4,"[55.92280471986926, 70.14478563067895, 99.7275...",sit
...,...,...
276,"[79.11447294534128, 97.12501634890181, 98.7461...",run
277,"[71.56505117707799, 26.56505117707799, 50.1944...",run
278,"[0.0, 349.695153531234, 95.71059313749963, 204...",run
279,"[132.51044707800085, 108.43494882292201, 65.77...",run


In [44]:
x = []
for row in data["angles"].values:
  x.append(row)

x = np.asarray(x)
y = data["action"].values

In [45]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((196, 96), (85, 96), (196,), (85,))

In [46]:
encoder=LabelEncoder()
y_train=encoder.fit_transform(y_train)
y_train=to_categorical(y_train)

y_test=encoder.fit_transform(y_test)
y_test=to_categorical(y_test)

In [47]:
def train_model(model, x_train, y_train, x_test,y_test):
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    model.fit(x_train, y_train, epochs=20)
    results = model.evaluate(x_test, y_test)

In [48]:
cnn = Sequential()
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
cnn.add(Conv1D(filters=15, kernel_size=3, activation="relu", input_shape=(x_train.shape[1], 1)))

cnn.add(MaxPool1D(pool_size=2, strides=2))
cnn.add(Conv1D(filters=15, kernel_size=3, activation="relu"))
cnn.add(MaxPool1D(pool_size=2, strides=2))
cnn.add(Flatten())

In [49]:
cnn.add(Dense(units=196,activation="relu"))
cnn.add(Dense(units=196,activation="sigmoid"))
cnn.add(Dense(units=7, activation="softmax"))

In [50]:
train_model(cnn, x_train, y_train, x_test,y_test)

Epoch 1/20
7/7 [==============================] - 1s 6ms/step - loss: 2.0793 - accuracy: 0.1684
Epoch 2/20
7/7 [==============================] - 0s 6ms/step - loss: 1.8000 - accuracy: 0.2755
Epoch 3/20
7/7 [==============================] - 0s 6ms/step - loss: 1.7466 - accuracy: 0.2857
Epoch 4/20
7/7 [==============================] - 0s 6ms/step - loss: 1.6779 - accuracy: 0.4388
Epoch 5/20
7/7 [==============================] - 0s 5ms/step - loss: 1.6407 - accuracy: 0.4286
Epoch 6/20
7/7 [==============================] - 0s 5ms/step - loss: 1.5726 - accuracy: 0.4796
Epoch 7/20
7/7 [==============================] - 0s 6ms/step - loss: 1.5575 - accuracy: 0.3980
Epoch 8/20
7/7 [==============================] - 0s 7ms/step - loss: 1.5038 - accuracy: 0.4847
Epoch 9/20
7/7 [==============================] - 0s 5ms/step - loss: 1.4609 - accuracy: 0.5000
Epoch 10/20
7/7 [==============================] - 0s 5ms/step - loss: 1.4232 - accuracy: 0.5153
Epoch 11/20
7/7 [======================